In [6]:
from MT5 import *
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import time
from sklearn.ensemble import VotingClassifier
import pickle
from joblib import dump, load
import os
from sklearn.preprocessing import StandardScaler

path = "" 


def create_model_weights(symbol):
    """ Weights for Linear regression on the percentage change"""
    # Import the data
    data = MT5.get_data(symbol, 3500)[["close"]].pct_change(1)
    
    # Create new variable
    data.columns = ["returns"]

    # Features engeeniring
    data["returns t-1"] = data[["returns"]].shift(1)

    # Mean of returns
    data["mean returns 15"] = data[["returns"]].rolling(15).mean().shift(1)
    data["mean returns 60"] = data[["returns"]].rolling(60).mean().shift(1)

    # Volatility of returns
    data["volatility returns 15"] = data[["returns"]].rolling(15).std().shift(1)
    data["volatility returns 60"] = data[["returns"]].rolling(60).std().shift(1)
    
    # Split the data
    data = data.dropna()
    split = int(0.80*len(data))
    
    # Train set creation
    X_train = data[["returns t-1", "mean returns 15", "mean returns 60",
              "volatility returns 15",
              "volatility returns 60"]].iloc[:split]
    y_train = np.round(data[["returns"]].iloc[:split]+0.5)
    
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    
    # Create the model
    tree = DecisionTreeClassifier(max_depth=6)
    svr = SVC()
    lin = LogisticRegression()

    alg = VotingClassifier(estimators=[
        ('lr', lin), ("tree", tree), ("svr", svr)])

    # Fit the model
    alg.fit(X_train, y_train)
    
    # Save the model
    alg_var = pickle.dumps(alg)
    alg_pickel = pickle.loads(alg_var)

    dump(alg_pickel ,os.path.join(path,f"Models/{symbol}_voting.joblib"))
    
    
    

def vot_cla_sig(symbol):
    """ Function for predict the value of tommorow using VOTING model"""
    
    # Create the weights if there is not in the folder
    try:
        alg = load(os.path.join(path,f"Models/{symbol}_voting.joblib"))
    except:
        create_model_weights(symbol)
        alg = load(os.path.join(path,f"Models/{symbol}_voting.joblib"))
    
    # Take the lastest percentage of change 
    data = MT5.get_data(symbol, 3500)[["close"]].pct_change(1)
    
    # Create new variable
    data.columns = ["returns"]

    # Features engeeniring

    # Mean of returns
    data["mean returns 15"] = data[["returns"]].rolling(15).mean()
    data["mean returns 60"] = data[["returns"]].rolling(60).mean()

    # Volatility of returns
    data["volatility returns 15"] = data[["returns"]].rolling(15).std()
    data["volatility returns 60"] = data[["returns"]].rolling(60).std()
    
    X = data[["returns", "mean returns 15", "mean returns 60",
              "volatility returns 15",
              "volatility returns 60"]].iloc[-1:,:].values
    
    # Find the signal
    prediction = alg.predict(X)
    prediction = np.where(prediction==0, -1, 1)
    buy = prediction[0] > 0
    sell = not buy
    
    
    return buy, sell


# True = Live Trading and False = Screener
live = True

if live:
    current_account_info = mt5.account_info()
    print("------------------------------------------------------------------")
    print("Date: ", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print(f"Balance: {current_account_info.balance} USD, \t"
          f"Equity: {current_account_info.equity} USD, \t"
          f"Profit: {current_account_info.profit} USD")
    print("------------------------------------------------------------------")


info_order = {
    "Microsoft": ["MSFT", 2],
    "Trex Co Inc": ["TREX", 96],
    #"Black Stone": ["BX", 48]
}


#start = datetime.now().strftime("%H:%M:%S")
start = "21:55:00"
while True:
    # Verfication for launch
    if datetime.now().weekday() not in (5,3):
        is_time = datetime.now().strftime("%H:%M:%S") == start #"23:59:59"
    else:
        is_time = False

    
    # Launch the algorithm
    if is_time:

        # Open the trades
        for asset in info_order.keys():

            # Initialize the inputs
            symbol = info_order[asset][0]
            lot = info_order[asset][1]

            # Create the signals
            buy, sell = vot_cla_sig(symbol)

             # Run the algorithm
            if live:
                MT5.run(symbol, buy, sell,lot)

            else:
                print(f"Symbol: {symbol}\t"
                     f"Buy: {buy}\t"
                     f"Sell: {sell}")
        time.sleep(1)

------------------------------------------------------------------
Date:  2023-05-09 20:23:18
Balance: 10005.31 USD, 	Equity: 9885.03 USD, 	Profit: -120.28 USD
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2023-05-09 20:23:18
SYMBOL: MSFT
BUY: True 	  SHORT: False
POSITION: 0 	 ID: 2017500830
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2023-05-09 20:23:18
SYMBOL: TREX
BUY: True 	  SHORT: False
POSITION: 0 	 ID: 2017518660
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2023-05-09 20:23:19
SYMBOL: MSFT
BUY: True 	  SHORT: False
POSITION: 0 	 ID: 2017500830
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2023-05-09 2

KeyboardInterrupt: 

In [12]:
start


'15:32:00'